# Time Evolution
Using what we learned from the effective Hamiltonian, it is now time to compute the density matrix. Recall that the density matrix (linearly mapped to a vector) absent any noise is given by:
$$
    \newcommand{ket}[1]{\left|#1\right\rangle}
    \newcommand{bra}[1]{\left\langle#1\right|}
    \vec{\rho}(t) = (R\otimes R)\left( \sum_{jk}e^{-i\omega_{jk}t}\ket{j}\bra{j}\otimes \ket{k}\bra{k} \right) (R^{-1}\otimes R^{-1})\vec{\rho}(0)
$$
with $R$ the eigenvector matrix and $\omega_{jk}$ the eigenvalues of the system.

## Single Donor, Single Photon
Again, the Hamiltonian in the single qubit case is:
$$
H = \hbar \omega_c a^\dagger a + \sum_j E_j \ket{j}\bra{j} + \frac{1}{2}g_c(a+a^\dagger)(1+Z)
$$
where $Z$ is defined as: 
$$
Z = \sum_{jk} z_{jk}\sigma_j\tau_j 
$$
$\omega_c$ is the cavity energy and $g_c$ is the photon-charge coupling strength. The eigenenergies up to second order in perturbation theory in the single qubit case are:
$$
        \begin{align}
            E_0 &= \frac{1}{2}(-\omega_0-\omega_B)-\frac{A}{8} \left(1-\cos \eta\right)-\frac{\Delta\omega_B}{4}\left(1+\cos \eta\right)-\frac{A^2}{16\omega_B}\left((1-\cos\eta)^2+\sin^2\eta(\frac{\omega_B}{4\omega_0}+\frac{\omega_B}{\omega_0+\omega_B}-\frac{\omega_B\Delta\omega_B}{A\omega_0}+\frac{\omega_B\Delta\omega_B^2}{A^2\omega_0})\right)\\
            E_1 &= \frac{1}{2}(-\omega_0+\omega_B)-\frac{A}{8} \left(1-\cos \eta\right)+\frac{\Delta\omega_B}{4}\left(1+\cos \eta\right)-\frac{A^2}{16\omega_B}\left(-(1-\cos\eta)^2+\sin^2\eta(\frac{\omega_B}{4\omega_0}+\frac{\omega_B}{\omega_0-\omega_B}+\frac{\omega_B\Delta\omega_B}{A\omega_0}+\frac{\omega_B\Delta\omega_B^2}{A^2\omega_0})\right) \\
            E_2 &= \frac{1}{2}( \omega_0-\omega_B)-\frac{A}{8} \left(1+\cos \eta\right)-\frac{\Delta\omega_B}{4}\left(1-\cos \eta\right)-\frac{A^2}{16\omega_B}\left((1+\cos\eta)^2+\sin^2\eta(-\frac{\omega_B}{4\omega_0}-\frac{\omega_B}{\omega_0-\omega_B}+\frac{\omega_B\Delta\omega_B}{A\omega_0}-\frac{\omega_B\Delta\omega_B^2}{A^2\omega_0})\right) \\
            E_3 &= \frac{1}{2}( \omega_0+\omega_B)-\frac{A}{8} \left(1+\cos \eta\right)+\frac{\Delta\omega_B}{4}\left(1-\cos \eta\right)-\frac{A^2}{16\omega_B}\left(-(1+\cos\eta)^2+\sin^2\eta(-\frac{\omega_B}{4\omega_0}-\frac{\omega_B}{\omega_0+\omega_B}-\frac{\omega_B\Delta\omega_B}{A\omega_0}-\frac{\omega_B\Delta\omega_B^2}{A^2\omega_0})\right)
        \end{align}
$$
and the coefficients of $Z$ are
$$
        \begin{align}
            z_{01} &= -A\omega_0\Delta\omega_B\cos\eta\sin^2\eta/4\omega_B(\omega_0^2-\omega_B^2) \\
            z_{03} &= A^2\omega_0^3\cos\eta\sin^2\eta/4\omega_B(\omega_0^2-\omega_B^2)^2 \\
            z_{10} &= \sin\eta + A\cos\eta\sin\eta/4\omega_0 \\
            z_{11} &= -A\omega_0\cos\eta\sin\eta/2(\omega_0^2-\omega_B^2) \\
            z_{13} &= -\Delta\omega_B\cos\eta\sin\eta/2\omega_0 \\
            z_{22} &= -A\omega_0^2\cos\eta\sin\eta/2\omega_B(\omega_0^2-\omega_B^2) \\
            z_{30} &= \cos\eta - A\sin^2\eta/4\omega_0 \\
            z_{31} &= A\omega_0\sin^2\eta/2(\omega_0^2-\omega_B^2) \\
            z_{33} &= \Delta\omega_B\sin^2\eta/2\omega_0 
        \end{align}
$$

In [6]:
# Import necessary libraries
import numpy as np
import pandas as pd
# Import Pauli Matrices and System Constants
from pauli import *
from constants import *

In [9]:
def FlipFlopEnergies(parameters):
    '''
        Returns the eigenenergies for the flip-flip qubit given above
        arguments:
            parameters{
                'Vt': tunnel coupling,
                'wB': zeeman splitting,
                'eps':applied electric field energy
            }
        returns:
            4x4 matrix with energies along the diagonal
    '''
    Vt = parameters['Vt']
    wB = parameters['wB']
    eps = parameters['eps']
    w0 = np.sqrt(eps**2 + Vt**2)
    eta = np.arctan2(Vt,eps)
    E_ff = np.zeros((4,4))
    E_ff[0,0] = -0.5*(w0+wB) - 0.125*hyperfine*(1-np.cos(eta))-0.25*Delta*wB*(1+np.cos(eta))-0.0625*(hyperfine**2/wB)*( (1-np.cos(eta))**2 + np.sin(eta)**2*( wB/(4*w0) + wB/(w0+wB) - Delta*wB**2/(hyperfine*w0) + Delta**2*wB**3/(w0*hyperfine**2) ))
    E_ff[1,1] = -0.5*(w0-wB) - 0.125*hyperfine*(1-np.cos(eta))+0.25*Delta*wB*(1+np.cos(eta))-0.0625*(hyperfine**2/wB)*(-(1-np.cos(eta))**2 + np.sin(eta)**2*( wB/(4*w0) + wB/(w0+wB) + Delta*wB**2/(hyperfine*w0) + Delta**2*wB**3/(w0*hyperfine**2) ))
    E_ff[2,2] =  0.5*(w0-wB) - 0.125*hyperfine*(1+np.cos(eta))-0.25*Delta*wB*(1-np.cos(eta))-0.0625*(hyperfine**2/wB)*( (1+np.cos(eta))**2 + np.sin(eta)**2*(-wB/(4*w0) - wB/(w0+wB) + Delta*wB**2/(hyperfine*w0) - Delta**2*wB**3/(w0*hyperfine**2) ))
    E_ff[3,3] =  0.5*(w0+wB) - 0.125*hyperfine*(1+np.cos(eta))+0.25*Delta*wB*(1-np.cos(eta))-0.0625*(hyperfine**2/wB)*(-(1+np.cos(eta))**2 + np.sin(eta)**2*(-wB/(4*w0) - wB/(w0+wB) - Delta*wB**2/(hyperfine*w0) - Delta**2*wB**3/(w0*hyperfine**2) ))
    return E_ff

def ElectronPositionCoefficients(parameters):
    '''
        Returns the coefficients z_jk for the flip-flip qubit given above
        arguments:
            parameters{
                'Vt': tunnel coupling,
                'wB': zeeman splitting,
                'eps':applied electric field energy
            }
        returns:
            4x4 matrix
    '''
    Vt = parameters['Vt']
    wB = parameters['wB']
    eps = parameters['eps']
    w0 = np.sqrt(eps**2 + Vt**2)
    eta = np.arctan2(Vt,eps)
    Zcoef = np.zeros((4,4))
    Zcoef[0,1] = 0
    Zcoef[0,3] = 0
    Zcoef[1,0] = 0
    Zcoef[1,1] = 0
    Zcoef[1,3] = 0
    Zcoef[2,2] = 0
    Zcoef[3,0] = 0
    Zcoef[3,1] = 0
    Zcoef[3,3] = 0
    return Zcoef

In [7]:
'''
    Units being used for analysis
        Energy: GHz
        Time  : ns
'''
wc = 0.5 # GHz, TODO: Find actual value for this
gc = 0.3 # GHz, TODO: Find actual value for this
